In [1]:
import numpy as np
from time import time
from driver import Model

In [2]:
myip = Model("bitstream/cnn.bit")

In [3]:
data = np.float32(np.load("test_x.npy").flatten())
gold = np.load("test_y.npy")

In [4]:
# sliding window with window size 15, stride 5
num_of_data = 30
data_length = 75 * 6
window_size = 15
window_stride = 5

highest_false_positive_confidence = 0

for i in range(0, num_of_data):
    print("-"*40 + f"Data: {i}" + "-"*40)
    data_frame = data[i*data_length:(i+1)*data_length]
    for j in range(0, data_length - window_size*6 + 1, window_stride*6):
        
        predicted, softmax_confidence = myip.inference(data_frame[j:j + 6*window_size])
        gold_result = np.argmax(gold[i])
        print(f"raw outputs: {myip.raw_outputs}")
        print(f"prediction: {predicted} vs GOLD: {gold_result}")
        print(f"confidence: {softmax_confidence * 100}%")
        print(f"remaining number of frames required to fill the window: {int(myip.register_map.data_required)}")
        print("\n")
        
        if predicted != np.argmax(gold[i]) and softmax_confidence > highest_false_positive_confidence:
            highest_false_positive_confidence = softmax_confidence
    myip.resetBuffer()

# if the confidence goes higher than this value then we do not need any more input
print(f"best threshold = {highest_false_positive_confidence}")

----------------------------------------Data: 0----------------------------------------
raw outputs: [-0.02001225  0.11897393  0.0389053 ]
prediction: 1 vs GOLD: 1
confidence: 35.800039768218994%
remaining number of frames required to fill the window: 12


raw outputs: [-0.19881305  0.05627504 -0.05766947]
prediction: 1 vs GOLD: 1
confidence: 37.49312460422516%
remaining number of frames required to fill the window: 11


raw outputs: [-0.1424431   0.14824043 -0.14267315]
prediction: 1 vs GOLD: 1
confidence: 40.07481932640076%
remaining number of frames required to fill the window: 10


raw outputs: [-0.1927769   0.20232359 -0.17405237]
prediction: 1 vs GOLD: 1
confidence: 42.3736572265625%
remaining number of frames required to fill the window: 9


raw outputs: [-0.16195258  0.20133552 -0.24442995]
prediction: 1 vs GOLD: 1
confidence: 42.8133487701416%
remaining number of frames required to fill the window: 8


raw outputs: [-0.22837569  0.07908447 -0.17298202]
prediction: 1 vs GOLD: 1